In [1]:
import numpy as np
from numba import jit
import torch

import timeit

In [2]:
def distances(x): # pytorch
    """Calculates distance vectors and distances (euclidian norm of vecs)
    
    Arguments:
        x (float): position vectors (dim = N x 3)
    
    Output:
        dist (float): distances between particle pairs (dim = N x N)
        vecs (float): distance vectors between particle pairs (dim = N x N x 3)
    """
    x = torch.Tensor(x)
    #dist = torch.zeros((x.shape[0], x.shape[0]))
    #vecs = torch.zeros((x.shape[0], x.shape[0], x.shape[-1]))
    vecs = x[None, :, :] - x[:, None, :]       
    return torch.norm(vecs, dim=-1), vecs

In [3]:
@jit
def Coulomb_force_jit(x, q):
    """Coulomb's law

    Arguments:
        x (float): position vectors (dim = N x 3)
        q (int): charges (dim = N)
        
    Constants:
        vacuum permittivity: eps0 = 8.854187e-12 
        elementary charge: qe = 1.602177e-19
    
    Output:
        f (float): forces between all particle pairs (dim = N x N x 3)
    """
    eps0, qe = 1., 1.
    force = np.zeros(x.shape)
    dist, vecs = distances(x)[0].numpy(), distances(x)[1].numpy()
    for i in range(x.shape[0]):
        for j in range(x.shape[0]):
            if dist[i][j] != 0:
                force[i] += q[i] * q[j] * vecs[i][j] / dist[i][j]**3
    return qe/(4*np.pi*eps0)*force

In [4]:
def Coulomb_force_vec(x, q):
    """Coulomb's law

    Arguments:
        x (float): position vectors (dim = N x 3)
        q (int): charges (dim = N)
        
    Constants:
        vacuum permittivity: eps0 = 8.854187e-12 
        elementary charge: qe = 1.602177e-19
    
    Output:
        f (float): forces between all particle pairs (dim = N x N x 3)
    """
    eps0, qe = 1., 1.
    force = np.zeros(x.shape)
    dist, vecs = distances(x)
    dist[dist!=0] = 1/dist[dist!=0]**3
    force = np.dot(np.diag(q), vecs * dist[:, :, None])
    force = np.einsum("ijk,j", force, q)
    return qe/(4*np.pi*eps0)*force

In [5]:
def coulomb(coord, q, eps0=1, pbc=False):
        dist, vectors = distances(coord)
        dist[dist!=0] = 1/dist[dist!=0]**3
        D = dist[:,:,None]*vectors
        return q[:, None]*np.einsum("ijk, j",D, q)

In [6]:
positions = np.array([[0., 0.],
                    [0., 1.],
                    [1., 1.],
                    [1., 0.]])
charges = np.array([1., -1., 1., -1.])

In [7]:
print(Coulomb_force_vec(positions, charges))
print(Coulomb_force_jit(positions, charges))

[[ 0.05144258  0.05144258]
 [ 0.05144258 -0.05144258]
 [-0.05144258 -0.05144258]
 [-0.05144258  0.05144258]]
[[-0.05144258 -0.05144258]
 [-0.05144258  0.05144258]
 [ 0.05144258  0.05144258]
 [ 0.05144258 -0.05144258]]


In [8]:
import random

def Random_particles(N):
    """Creates a list of N particles with random positions and charges
    
    Arguments:
        N (int): number of particles
        
    Output:
        x (float): position vectors (dim = N x 3)
        q (int): charges (dim = N)
    """
    return np.random.rand(N,3), np.array([[-1,1][random.randrange(2)] for i in range(N)])

In [9]:
N = 100
x, q = Random_particles(N)

In [10]:
%timeit Coulomb_force_vec(x, q)
%timeit Coulomb_force_jit(x, q)

8.98 ms ± 304 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.54 ms ± 517 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit coulomb(x, q)

4.02 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
